**1801CS37 - ENDSEM ASSIGNMENT**


In [1]:
from queue import PriorityQueue
from dataclasses import dataclass
import random
import time
import math
import fileinput

optimal_path = []
MAX_ITERATIONS = 100000
outFile = open("output.txt", "w")

Node Class

In [2]:
@dataclass(order=True)
class Node():
    """Node class :-
        state: board configuration(a list)
        depth: Node depth - g(n)
        h1: Number of tiles displaced from their destined position
        h2: Sum of Manhattan distance of each tile from the goal position
        parent: parent of the current node 
    """
    def __init__(self, state, depth):
        self.state = state
        self.depth = depth
        self.h1 = None
        self.h2 = None
        self.parent = None
        self.priority = random.randrange(1)

    def calc_heuristic(self, goal_state):
        self.h1 = h1_function(self.state, goal_state)
        self.h2 = h2_function(self.state, goal_state)
        self.h3 = self.h1 + self.h2

    def back_track(self):
        current = self
        count = 0
        optimal_path.clear()
        while (current != None):
            bs = board_state(current.state)
            optimal_path.append(current.state)
            count += 1
            current = current.parent
        return count

Heuristics

In [3]:
def h1_function(current_state, goal_state):
    """Number of tiles displaced from their destined position."""
    cost=0
    # print(current_state, goal_state)
    for i in range(len(current_state)):
        if current_state[i]!=goal_state[i]:
            cost+=1
    return cost

def h2_function(current_state, goal_state): 
    """Sum of Manhattan distance of each tile from the goal position."""
    cost=0
    final_position = cordinates(goal_state)
    temp=board_state(current_state)
    for i in range(3):
        for j in range(3):
            t=temp[i][j]
            xf, yf = final_position[t]
            cost += abs(xf-i)+abs(yf-j)
    return cost

Utility Functions

In [4]:
def board_state(state):
    """Return 2-d matrix representation"""
    i=0
    temp=[([0]*3) for j in range(3)]
    for row in range(3):
        for col in range(3):
            temp[row][col]=state[i]
            i+=1
    return temp

def display_board(state, outFile):
    """Print the board"""
    outFile.write("\n-------------")
    outFile.write("\n| %i | %i | %i |" % (state[0], state[1], state[2]))
    outFile.write("\n-------------")
    outFile.write("\n| %i | %i | %i |" % (state[3], state[4], state[5]))
    outFile.write("\n-------------")
    outFile.write("\n| %i | %i | %i |" % (state[6], state[7], state[8]))
    outFile.write("\n-------------\n")

def cordinates(state):
    """Return position coordinates in the goal state"""
    cords = [None] * 9
    for index, i  in enumerate(state):
        cords[i] = (index // 3, index % 3)
    return cords

# move the blank title up on the board
def move_up(state):
    new_state=state[:]
    index = new_state.index(0)
    if index not in [0,1,2]:
        temp = new_state[index-3]
        new_state[index-3]=new_state[index]
        new_state[index]=temp
    return new_state

# move the blank title down on the board
def move_down(state):
    new_state=state[:]
    index=new_state.index(0)
    if index not in [6,7,8]:
        temp = new_state[index+3]
        new_state[index+3]=new_state[index]
        new_state[index]=temp
    return new_state

# move the blank title left on the board
def move_left(state):
    new_state = state[:]
    index = new_state.index(0)
    if index not in [0,3,6]:
        temp = new_state[index-1]
        new_state[index-1]=new_state[index]
        new_state[index]=temp
    return new_state

# move the blank title right on the board
def move_right(state):
    new_state = state[:]
    index = new_state.index(0)
    if index not in [2,5,8]:
        temp = new_state[index+1]
        new_state[index+1] = new_state[index]
        new_state[index]=temp
    return new_state

In [5]:
def expansion(state):
    """Expand nodes along the children"""
    expanded_nodes = []
    expanded_nodes.append(move_up(state))       # moving up
    expanded_nodes.append(move_down(state))     # moving down
    expanded_nodes.append(move_left(state))     # moving left
    expanded_nodes.append(move_right(state))    # moving right
    expanded_nodes = [x for x in expanded_nodes if x]
    return(expanded_nodes)

Hill Climbing

In [6]:
def hillClimbing(start_state, goal_state, heuristic):
    """Cost function: f(n) = h(n). In this algorithm, we push the child nodes 
    corresponding to the least heuristic greedily and discard the others.
    Might not result in global optimum path"""
    iterations = 0
    open = PriorityQueue()
    closed = []

    start_node = Node(start_state, 0)
    start_node.calc_heuristic(goal_state)
    
    open.put((getattr(start_node, heuristic), start_node))

    while (not open.empty()):
        iterations+=1
        if iterations>MAX_ITERATIONS:
            print("Failure: No solution found")
            print("Total number of states explored:\n", len(closed))
            return None
        cost, parent = open.get()
        open.queue.clear()
        closed.append(parent.state)
        if (parent.state == goal_state):
            return (parent, len(closed))
        for i in expansion(parent.state):
            temp_node = Node(i, parent.depth + 1)
            temp_node.parent = parent

            if (temp_node.state == goal_state):
                return (temp_node, len(closed))
            elif (temp_node.state in closed):
                continue
            
            temp_node.calc_heuristic(goal_state)
            open.put((getattr(temp_node, heuristic), temp_node))

Driver code

In [7]:
def main():
    # Read input
    lines = fileinput.FileInput(files = 'input.txt')

    # Define start, goal state
    start_state = [int(x) for x in lines[0].split(' ')]
    goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]

    # Start Hill Climbing Algorithm
    outFile.write("\nHill Climb Search:\n\n")
    

    heuristic_types = ["h1", "h2", "h3"]
    for i in range(3):
        start_clock = time.time()
        heuristic = heuristic_types[i]
        outFile.write("Heuristic: " + heuristic + "\n\n")

        val = hillClimbing(start_state, goal_state, heuristic)
        
        # If Valid solution, display it
        if val!=None:
            node_1 = val[0]
            expl = val[1]
            
            outFile.write("Success! Solution found \n")
            hc_execution_time = time.time()-start_clock
            
            outFile.write("Total number of states explored:" + str(expl) + "\n")
            optimal_cost = node_1.back_track()
            
            outFile.write("Total number of states in optimal path:" + str(optimal_cost) + "\n")
            outFile.write("\n\nOptimal Path: \n")

            optimal_path.reverse()
            for state in optimal_path:
                display_board(state, outFile)
    
            outFile.write("Optimal path cost:" + str(optimal_cost) + "\n")
            outFile.write("Time taken for execution (Hill Climbing):" + str(hc_execution_time) + "\n")
        # If invalid, print no solution
        else:
            outFile.write("\nNo solution found\n")
        
        outFile.write("\n\n\n------------------------------------------------------\n\n")

    outFile.close()

if __name__ == "__main__":
    main()